In [0]:
%pip install -U -qqqq "requests>=2.32.3", "databricks-agents>=0.13.0", "databricks-sdk[openai]>=0.40.0", "backoff>=2.2.1", "mlflow"
dbutils.library.restartPython()

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
import simulator_service
import logging
from typing import Callable, Dict, Any, List
from simulator_service import SyntheticDataSimulatorService
from pathlib import Path
import mlflow
from mlflow.models.resources import DatabricksServingEndpoint
from simulator_service.mlflow_utils import invoke_model_with_trace
from simulator_service.context_generators import (
    get_all_tool_outputs_from_agent_trace,
    get_agent_response_from_trace,
)
from simulator_service.synthetic_generation import (
    generate_next_question_using_context_from_previous_turn,
)

# Setup logger
logger = logging.getLogger("simulator_service.examples")

# Helpers

## Agent MLflow logging helper

In [0]:

def log_model(
    agent_code_file: str, agent_config: dict
) -> mlflow.models.model.ModelInfo:
    """Log the model to MLflow and return the model info.

    Args:
        agent_code_file: Path to the agent code file
        agent_config: Configuration dictionary for the agent

    Returns:
        ModelInfo: Information about the logged model
    """
    return mlflow.pyfunc.log_model(
        python_model=agent_code_file,
        artifact_path="agent",
        model_config=agent_config,
        resources=[
            DatabricksServingEndpoint(endpoint_name=agent_config["endpoint_name"])
        ],
        input_example={
            "messages": [{"role": "user", "content": "What is lakehouse monitoring?"}]
        },
        pip_requirements=[
            "databricks-sdk[openai]",
            "mlflow",
            "databricks-agents",
            "backoff",
        ],
    )

## Wrapper function so the synthetic service can call the logged Agent

See: https://github.com/epec254/synthetic-simulator?tab=readme-ov-file#required-callable-signatures

In [0]:
def get_agent_callable(
    model_info: mlflow.models.model.ModelInfo,
) -> Callable[[List[Dict[str, str]]], Dict[str, Any]]:
    """
    Build a chat completion function that uses a logged MLflow model.
    This is passed to `SyntheticDataSimulatorService(chat_agent_callable=...)`

    Args:
        model_info: ModelInfo object returned from log_model()

    Returns:
        Callable that implements chat completion API using the logged model
    """
    # Load the model once when creating the completion function
    loaded_model = mlflow.pyfunc.load_model(model_info.model_uri)

    def call_mlflow_logged_agent(messages: List[Dict[str, str]]) -> Dict[str, Any]:
        # print(messages)
        # Use invoke_model_uri_with_trace with the cached model's URI
        outputs, output_trace = invoke_model_with_trace(
            model=loaded_model,
            model_input={"messages": messages},
        )

        return {
            "outputs": outputs,
            "output_trace": output_trace,
        }

    return call_mlflow_logged_agent



# Synthetic generation

## Approach #1: Use the agent's tool outputs to synthesize the next turn of conversation.

In [0]:
def generate_based_on_tool_outputs(
    max_turns: int,
    model_info: mlflow.models.model.ModelInfo,
    output_file: str,
    seed_question: str,
    agent_description: str,
    tag: str,
):
    """
    Synthetically generates follow-up questions based on the outputs from all called tools.

    Args:
        max_turns: Maximum number of conversation turns
        model_info: ModelInfo object returned from log_model()
        output_file: Path to the output file for conversation history
        seed_question: Initial question to start the conversation
        agent_description: Description of the chat agent
        tag: Tag for the generation type
    """

    logger.info(
        f"Starting synthetic generation with {max_turns} turns based on tool outputs"
    )

    chat_completion_callable = get_agent_callable(model_info)

    simulator_service = SyntheticDataSimulatorService(
        chat_agent_callable=chat_completion_callable,
        question_generator_callable=generate_next_question_using_context_from_previous_turn,
        get_context_from_chat_agent_response_for_next_turn_callable=get_all_tool_outputs_from_agent_trace,
        max_turns=max_turns,
        seed_question=seed_question,
        output_file=output_file,
        agent_description=agent_description,
        tag=tag,
    )

    # Start the conversation
    try:
        simulator_service.start_conversation()
    except Exception as e:
        logger.error(f"Error during conversation: {e}")
        raise


## Approach #2: Use the agent's response to synthesize the next turn of conversation.

In [0]:
def generate_based_on_response(
    max_turns: int,
    model_info: mlflow.models.model.ModelInfo,
    output_file: str,
    seed_question: str,
    agent_description: str,
    tag: str,
):
    """
    Synthetically generates follow-up questions based on the agent's last response.

    Args:
        max_turns: Maximum number of conversation turns
        model_info: ModelInfo object returned from log_model()
        output_file: Path to the output file for conversation history
        seed_question: Initial question to start the conversation
        agent_description: Description of the chat agent
        tag: Tag for the generation type
    """

    logger.info(
        f"Starting synthetic generation with {max_turns} turns based on response content"
    )

    chat_completion_callable = get_agent_callable(model_info)

    simulator_service = SyntheticDataSimulatorService(
        chat_agent_callable=chat_completion_callable,
        question_generator_callable=generate_next_question_using_context_from_previous_turn,
        get_context_from_chat_agent_response_for_next_turn_callable=get_agent_response_from_trace,
        max_turns=max_turns,
        seed_question=seed_question,
        output_file=output_file,
        agent_description=agent_description,
        tag=tag,
    )

    # Start the conversation
    try:
        simulator_service.start_conversation()
    except Exception as e:
        logger.error(f"Error during conversation: {e}")
        raise

## Run the simulations

In [0]:
import os


# Create output directory
output_dir = Path("output")
output_dir.mkdir(exist_ok=True)
output_file = str(output_dir / "synthetic_evaluation_set.jsonl")

# Log the model
# We use a function-calling agent defined in fc_agent.py that queries databricks documentation through a keyword-based retriever tool.
# You can replace this with your own agent implementation.

# Note: this configuration is specific to the code in the `fc_agent.py` file.  Your own agent will likely have a different configuration.
agent_config = {
    "endpoint_name": "ep-gpt4o-new",  # replace with a Model Serving endpoint that supports Chat Completions.  Can be an external model e.g., OpenAI
    "temperature": 0.01,
    "max_tokens": 1000,
    "system_prompt": """You are a helpful assistant that answers questions about Databricks. Questions unrelated to Databricks are irrelevant.

You answer questions using a set of tools. If needed, you ask the user follow-up questions to clarify their request.
""",
    "max_context_chars": 4096 * 4,
}
model_info = log_model(
    agent_code_file=str(os.getcwd()+ "/fc_agent.py"),
    agent_config=agent_config,
)

# Common parameters

seed_questions = ["what is lakehouse monitoring?"] # Add more questions to this array

params = {
    "max_turns": 2,
    "model_info": model_info,
    "output_file": output_file,
    "agent_description": "A chat agent that answers questions about Databricks documentation.",
}

# TODO: Parraellize
for seed_question in seed_questions:
  # Run both types of generation with different tags
  generate_based_on_tool_outputs(**params, tag="tool_outputs", seed_question=seed_question)
  generate_based_on_response(**params, tag="agent_response", seed_question=seed_question)



2025/01/13 15:12:46 INFO mlflow.pyfunc: Predicting on input example to validate output


Uploading artifacts:   0%|          | 0/12 [00:00<?, ?it/s]

2025-01-13 15:13:09,411 - chat_service - INFO - Initial turn: Asking seed question: what is lakehouse monitoring?


INFO:chat_service:Initial turn: Asking seed question: what is lakehouse monitoring?


2025-01-13 15:13:20,413 - chat_service - INFO - Answer: Lakehouse monitoring in Databricks refers to the capabilities that allow you to monitor the quality of data in all tables within your account, as well as track the performance of machine learning models and model-serving endpoints. This includes setting up alerts to notify you when certain metrics or statistics move out of a specified range, indicating potential issues such as data drift or the need for model retraining.

For more detailed information, you can refer to the [Databricks Lakehouse Monitoring documentation](https://docs.databricks.com/lakehouse-monitoring/monitor-alerts.html).


...

INFO:chat_service:Answer: In Databricks Lakehouse, you can configure monitor alerts to send notifications through various channels, including:

1. Email
2. Webhooks
3. Applications like Slack
4. PagerDuty

These channels allow you to receive alerts when certain metrics move out of a specified range or when there are significant changes in your data.


2025-01-13 15:14:07,872 - chat_service - INFO - Turn 2: Asking question: What channels can you configure for Databricks Lakehouse monitor alerts?


INFO:chat_service:Turn 2: Asking question: What channels can you configure for Databricks Lakehouse monitor alerts?


2025-01-13 15:14:13,866 - chat_service - INFO - Answer: I couldn't find specific documentation on the notification channels for Databricks Lakehouse monitor alerts. However, typically, you can configure alerts to send notifications via email, webhooks, and applications like Slack and PagerDuty. If you need more detailed or specific information, I recommend checking the official Databricks documentation or contacting their support.


INFO:chat_service:Answer: I couldn't find specific documentation on the notification channels for Databricks Lakehouse monitor alerts. However, typically, you can configure alerts to send notifications via email, webhooks, and applications like Slack and PagerDuty. If you need more detailed or specific information, I recommend checking the official Databricks documentation or contacting their support.


## View generated data

In [0]:
import pandas as pd

synthetic_questions_df = pd.read_json(output_file, orient="records", lines=True)
display(synthetic_questions_df)

request response metadata List(List(You are a helpful AI assistant engaging in a conversation., system), List(what is lakehouse monitoring?, user)) List(Lakehouse monitoring in Databricks refers to the capabilities that allow you to monitor the quality of data in all tables within your account, as well as track the performance of machine learning models and model-serving endpoints. This includes setting up alerts to notify you when certain metrics or statistics move out of a specified range, indicating potential issues such as data drift or the need for model retraining.

For more detailed information, you can refer to the [Databricks Lakehouse Monitoring documentation](https://docs.databricks.com/lakehouse-monitoring/monitor-alerts.html)., assistant) List(null, true, what is lakehouse monitoring?, tool_outputs) List(List(You are a helpful AI assistant engaging in a conversation., system), List(what is lakehouse monitoring?, user), List(Lakehouse monitoring in Databricks refers to the capabilities that allow you to monitor the quality of data in all tables within your account, as well as track the performance of machine learning models and model-serving endpoints. This includes setting up alerts to notify you when certain metrics or statistics move out of a specified range, indicating potential issues such as data drift or the need for model retraining.

For more detailed information, you can refer to the [Databricks Lakehouse Monitoring documentation](https://docs.databricks.com/lakehouse-monitoring/monitor-alerts.html)., assistant), List(How can I receive notifications for Databricks Lakehouse monitor alerts?, user)) List(To receive notifications for Databricks Lakehouse monitor alerts, you can set up alerts using Databricks SQL. Here's how you can do it:

1. **Create a Databricks SQL Query**: First, create a SQL query on the monitor profile metrics table or drift metrics table. This query will define the conditions under which you want to be alerted.

2. **Create a Databricks SQL Alert**: Once you have your query, create an alert for it. You can configure the alert to evaluate the query at a desired frequency.

3. **Configure Notifications**: By default, email notifications are sent when an alert is triggered. However, you can also set up notifications to be sent to other applications such as Slack or PagerDuty using webhooks.

4. **Quick Alert Creation from Dashboard**: You can also create an alert directly from the monitor dashboard:
 - Find the chart for which you want to create an alert.
 - Click the kebab menu (three vertical dots) in the upper-right corner of the chart and select "View query."
 - In the SQL editor, click the kebab menu above the editor window and select "Create alert."
 - Configure the alert and click "Create alert."

For more detailed steps and options, you can refer to the [Databricks Lakehouse Monitoring documentation](https://docs.databricks.com/lakehouse-monitoring/monitor-alerts.html)., assistant) List([{"page_content": "# Introduction to Databricks Lakehouse Monitoring\n", "metadata": {"doc_uri": "https://docs.databricks.com/lakehouse-monitoring/monitor-alerts.html", "score": 2, "chunk_id": "46a5182ce48b20d7617a750390b29baa"}}, {"page_content": "# Introduction to Databricks Lakehouse Monitoring\n### Monitor alerts\n\nPreview \nThis feature is in [Public Preview](https://docs.databricks.com/release-notes/release-types.html). \nThis page describes how to create a Databricks SQL alert based on a metric from a monitor metrics table. Some common uses for monitor alerts include: \n* Get notified when a statistic moves out of a certain range. For example, you want to receive a notification when the fraction of missing values exceeds a certain level.\n* Get notified of a change in the data. The drift metrics table stores statistics that track changes in the data distribution.\n* Get notified if data has drifted in comparison to the baseline table. You can set up an alert to investigate the data changes